In [ ]:
from src.material.material.phong_material import *
from src import Sphere
from src import PointLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src import ipynb_display_images
from src.material.textures.slope_map import SlopeMap
from src.render.loops import *
from src import Plane

# Simple scene with a sphere and a plane and a light source
no skybox just to demonstrate basic functionality

In [ ]:
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from math import sin, cos, pi

from src.material.material.phong_material import PhongMaterial
from src.material.textures.bump_map import BumpMap
from src.material.textures.slope_map import SlopeMap, SlopeMapPoint
from src.material.color import Color

from src.geometry.objects.sphere import Sphere
from src.geometry.objects.box import Box

from src.scene.primitive import Primitive
from src.scene.scene import Scene
from src.scene.camera import Camera
from src.scene.light import PointLight, AmbientLight

from src.math import Vertex, Vector
from src.render.resolution import Resolution


# ---------------------------------------------------------------------------
# 1) BUMP HEIGHT FUNCTIONS (POV-Ray style)
# ---------------------------------------------------------------------------

# Egg-carton style bump
def height_egg(u: float, v: float) -> float:
    return 0.5 * (sin(4 * 2*pi*u) + sin(4 * 2*pi*v))

egg_bump = BumpMap(
    height=height_egg,
    strength=0.35,
    scale_u=1.0,
    scale_v=1.0,
)

# Concentric ring bump (nice on spheres)
def height_rings(u: float, v: float) -> float:
    r = ((u - 0.5)**2 + (v - 0.5)**2) ** 0.5
    return sin(8 * 2*pi * r)

rings_bump = BumpMap(
    height=height_rings,
    strength=0.35,
    scale_u=1.0,
    scale_v=1.0,
)

# Soft horizontal “waves” (looks good on boxes/floors)
def height_waves(u: float, v: float) -> float:
    return 0.3 * sin(3 * 2*pi * v)

waves_bump = BumpMap(
    height=height_waves,
    strength=0.1,
    scale_u=1.0,
    scale_v=1.0,
)


# ---------------------------------------------------------------------------
# 2) SLOPE MAP – macro modulation along v (e.g. stronger distortion at equator)
# ---------------------------------------------------------------------------

equator_slope = SlopeMap.from_pairs([
    (0.0, Vector(0.0, 0.0, 0.0)),    # pole: no macro tilt
    (0.45, Vector(0.2, 0.0, 0.0)),
    (0.5, Vector(0.5, 0.0, 0.0)),    # equator: strongest
    (0.55, Vector(0.2, 0.0, 0.0)),
    (1.0, Vector(0.0, 0.0, 0.0)),    # other pole: no tilt
])


# ---------------------------------------------------------------------------
# 3) MATERIALS
# ---------------------------------------------------------------------------

# Reference smooth red (no bump)
smooth_red = PhongMaterial(
    name="smooth_red",
    base_color=Color.custom_rgb(255, 40, 40),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=80,
    reflectivity=0.25,
    transparency=0.0,
    slope_map=None,
    bump_map=None,
)

# Egg-carton bump
bumpy_red_egg = PhongMaterial(
    name="bumpy_red_egg",
    base_color=Color.custom_rgb(255, 40, 40),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=80,
    reflectivity=0.25,
    transparency=0.0,
    slope_map=None,
    bump_map=egg_bump,
)

# Rings bump + macro slope at equator
bumpy_red_rings = PhongMaterial(
    name="bumpy_red_rings",
    base_color=Color.custom_rgb(255, 70, 70),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=60,
    reflectivity=0.25,
    transparency=0.0,
    slope_map=equator_slope,
    bump_map=rings_bump,
)

# Ground: slightly wavy concrete
ground_material = PhongMaterial(
    name="ground_grey",
    base_color=Color.custom_rgb(130, 130, 130),
    spec_color=Color.custom_rgb(90, 90, 90),
    shininess=20,
    reflectivity=0.08,
    transparency=0.0,
    slope_map=None,
    bump_map=waves_bump,
)

# A tall bumpy column using egg bump
column_material = PhongMaterial(
    name="column_grey",
    base_color=Color.custom_rgb(200, 200, 200),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=40,
    reflectivity=0.12,
    transparency=0.0,
    slope_map=None,
    bump_map=egg_bump,
)


# ---------------------------------------------------------------------------
# 4) CAMERA + LIGHTS + SKYBOX
# ---------------------------------------------------------------------------

camera = Camera(
    fov=45,
    aspect_ratio=Resolution.R480p.aspect_ratio,
    origin=Vertex(0, 0.8, 1.8),
    direction=Vector(0, -0.05, -1).normalize_ip(),
)

key_light = PointLight(
    position=Vertex(3, 5, 3),
    intensity=600.0,
    falloff=0.02,
)

fill_light = PointLight(
    position=Vertex(-4, 3, 1),
    intensity=250.0,
    falloff=0.03,
)

ambient = AmbientLight(
    intensity=0.18,
)

scene = Scene(
    camera=camera,
    lights=[key_light, fill_light, ambient],
    skybox_path="./skybox/shanghai_4k.hdr",  # reflections from HDR
)


# ---------------------------------------------------------------------------
# 5) GEOMETRY (3 spheres + ground + column)
# ---------------------------------------------------------------------------

primitives: list[Primitive] = [
    # Ground platform
    Primitive(
        geometry=Box(
            corner1=Vertex(-4.0, -1.2, -7.0),
            corner2=Vertex( 4.0, -1.0, -2.0),
        ),
        material=ground_material,
    ),

    # Left: smooth reference sphere
    Primitive(
        geometry=Sphere(center=Vertex(-2.0, -0.1, -4.2), radius=1.0),
        material=smooth_red,
    ),

    # Center: egg-carton bump sphere
    Primitive(
        geometry=Sphere(center=Vertex(0.0, -0.1, -4.5), radius=1.0),
        material=bumpy_red_egg,
    ),

    # Right: rings + slope-modulated bumps
    Primitive(
        geometry=Sphere(center=Vertex(2.0, -0.1, -4.2), radius=1.0),
        material=bumpy_red_rings,
    ),

    # Tall bumpy column in the back-right
    Primitive(
        geometry=Box(
            corner1=Vertex(2.8, -1.0, -6.5),
            corner2=Vertex(3.4,  1.5, -5.5),
        ),
        material=column_material,
    ),
]

scene.add_primitives(primitives=primitives)
scene.validate()

render_configuration = RenderConfig(
    resolution=Resolution.R480p,
    samples_per_pixel=8,
    max_depth=10,
)
preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

# Now we can render the scene

In [ ]:
my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration)
png_path = my_ray_tracer.render("images/my_first_render.png")
ipynb_display_images(png_path)